In [1]:
import gurobipy as gp

In [2]:
from gurobipy import GRB
from gurobipy import quicksum

In [3]:
m = gp.Model()

Set parameter Username
Academic license - for non-commercial use only - expires 2022-02-12


In [4]:
Y = {}
for i in range(1,8):
    Y[i] = m.addVar(vtype=GRB.BINARY, name="Y"+str(i))

In [5]:
I = {}
for i in range(1,8):
    I[i] = m.addVar(vtype=GRB.CONTINUOUS, name="I"+str(i))
    
D = {}
for i in range(1,8):
    D[i] = m.addVar(vtype=GRB.CONTINUOUS, name="D"+str(i))
    
X = {}
for i in range(1,8):
    X[i] = m.addVar(vtype=GRB.CONTINUOUS, name="X"+str(i))

A = m.addVar(vtype=GRB.BINARY, name="A")
B = m.addVar(vtype=GRB.BINARY, name="B")
C = m.addVar(vtype=GRB.BINARY, name="C")
E = m.addVar(vtype=GRB.BINARY, name="E")

In [6]:
P = {1:1, 2:0, 3:1, 4:0, 5:0, 6:0, 7:0, 8:0, 9:1}
QOL_P_parameters = {1:-5, 2:-0.5, 3:-12, 4:-8, 5:-5, 6:-5, 7:-1, 8:-3, 9:-2}
QOL_Y_parameters = {1:-5, 2:-6, 3:-4, 4:-4, 5:-8, 6:-6, 7:-7}
QOL_X_parameters = {1:0.28, 2:0.30, 3:0.25, 4:0.17, 5:0.31, 6:0.246, 7:0.40}

minDosages = {1:20, 2:10, 3:20, 4:10, 5:10, 6:20, 7:20}
maxDosages = {1:80, 2:50, 3:100, 4:100, 5:70, 6:90, 7:50}
baseRegimenDosages = {1:20, 2:0, 3:30, 4:15, 5:0, 6:0, 7:35}
totalDosageCl = 275

Q_threshold = 35

fixedCosts = {1:25, 2:50, 3:10, 4:25, 5:20, 6:30, 7:40}
unitCosts = {1:1, 2:2, 3:1, 4:3, 5:2, 6:1, 7:1}
 
M = 10000

In [7]:
isDrugInBaseRegimen = {1:True, 2:False, 3:True, 4:True, 5:False, 6:False, 7:True}
m.setObjective( quicksum([(fixedCosts[i] * (1 - Y[i]) if isDrugInBaseRegimen[i] else fixedCosts[i] * Y[i]) for i in range(1,8)])
               + quicksum([unitCosts[i] * (I[i] + D[i]) for i in range(1,8)])
               , GRB.MINIMIZE)

In [8]:
helperXConstraints = {}
minDrugConstraint = {}
maxDrugConstraint = {}
for i in range(1,8):
    helperXConstraints[i] = m.addConstr(X[i] == baseRegimenDosages[i] + I[i] - D[i])
    minDrugConstraint[i] = m.addConstr(X[i] >= minDosages[i] * Y[i])
    maxDrugConstraint[i] = m.addConstr(X[i] <= maxDosages[i] * Y[i])
    
totalDosageConstraint = m.addConstr(quicksum([X[i] for i in range(1,8)]) == totalDosageCl)

QOFConstraint = m.addConstr(quicksum([QOL_P_parameters[i] * P[i] for i in range(1,10)])
                            + quicksum([QOL_Y_parameters[i] * Y[i] for i in range(1,8)])
                            + quicksum([QOL_X_parameters[i] * X[i] for i in range(1,8)])
                            >= Q_threshold
)

extraContstraint1_1 = m.addConstr(Y[1] + Y[2] - 1 <= M * A)
extraContstraint1_2 = m.addConstr(50 - X[1] - X[2] <= M * (1 - A))

extraContstraint2_1 = m.addConstr(Y[1] + Y[2] - 1 <= M * B)
extraContstraint2_2 = m.addConstr(X[1] + X[2] - 70 <= M * (1 - B))

extraContstraint3_1 = m.addConstr(1 - Y[5] <= M * C)
extraContstraint3_2 = m.addConstr(X[3] - 25 <= M * (1 - C))

extraContstraint4_1 = m.addConstr(Y[4] + Y[6] - 1 <= M * E)
extraContstraint4_2 = m.addConstr(1 - Y[5] - Y[7] <= M * (1 - E))

In [9]:
m.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 31 rows, 32 columns and 92 nonzeros
Model fingerprint: 0x1c9c3aad
Variable types: 21 continuous, 11 integer (11 binary)
Coefficient statistics:
  Matrix range     [2e-01, 1e+04]
  Objective range  [1e+00, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]
Presolve removed 4 rows and 11 columns
Presolve time: 0.00s
Presolved: 27 rows, 21 columns, 78 nonzeros
Variable types: 11 continuous, 10 integer (10 binary)
Found heuristic solution: objective 425.7812500

Root relaxation: objective 2.349547e+02, 11 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  234.95474    0    2  425.78125  234.95474  44.8%     -    0s
H    0     0                     325.0000000

In [10]:
m.printAttr('X')


    Variable            X 
-------------------------
          Y1            1 
          Y3            1 
          Y4            1 
          Y5            1 
          Y7            1 
          I1           60 
          I3       34.375 
          I5           70 
          I7           15 
          D4        4.375 
          X1           80 
          X3       64.375 
          X4       10.625 
          X5           70 
          X7           50 
           E            1 
